# Getting started with neural networks: classification and regression

## 영화 평 분류 : 이진 분류(binary classification) 예

### IMDB 영화평 데이터베이스: Tensorflow 의 Keras 라이브러리가 기본으로 제공하는 영화평 데이터베이스

**IMDB 데이터세트 로딩: IMDB 는 Internet Movide Database 로 25,000 개의 영화 리뷰가 training 으로, 그리고 25,000 개의 또 다른 
영화 리뷰가 testing 으로 구성되며 각각의 데이터세트는 50% 가 긍정, 50% 부정 평가를 담고 있다.

- num_words = 10000 변수는 trianing 데이터 중에 상위 10,000 개의 빈번하게 발생하는 단어들만을 유지하겠다는 의미로, 만약 이러한 제한을 설정하지 않으면 88,585 개의 독립적인 단어들을 다루게되므로 불필요하게 큰 용량과 씨름하게 됨
- 이러한 단어들은 한 샘플에서 한번만 나타날 수도 있는 것으로, 분류에 유용하지 않을 수 있음

In [ ]:
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)


- train_data 의 영화 평은 단어 index 들의 리스트로 이루어 짐

In [ ]:
train_data[0]

- train_label 과 test_label은 0 과 1 의 리스트로 구성. 0은 부정, 1은 긍정 표현

In [ ]:
train_labels[0]

- 상위 10,000 개의 자주 발생하는 단어들로 제한했기때문에, 단어 index 가 10,000 을 초과할 수 없다

In [ ]:
max([max(sequence) for sequence in train_data])

### 인코딩되어있는 영화평을 영어 단어의 영화 리뷰로 디코딩
- word_index 는 단어를 정수 인덱스로 dictionary 형태로 mapping
- reverse_word_index 를 통해 index 를 단어로 mapping
- decode_review 에서 3개의 index 들은 시스템 내부에서 예약되어 있음: 0: "padding", 1: "start of sequence", 2: "unknown"


In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()])
decoded_review = " ".join(
    [reverse_word_index.get(i - 3, "?") for i in train_data[0]])

### 데이터 준비
- 리스틀 tensor 로 신경망에 주입
- 정수 리스트를 0 과 1 로 이루어진 one-hot vector 로 변환하여 인코딩
- 맨처음 results 를 0 으로 채워진 행렬을 생성
- "results[i, sequence] = 1" 를 통하여 특정 index 의 results 를 1 로 셋팅
- train 과 test 데이터를 벡터화

In [ ]:
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

- one-hot-encoding 된 결과

In [ ]:
x_train[0]

- 레이블을 벡터화

In [ ]:
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

- 이제 데이터는 신경망에 투입될 수 있게 준비 됨

### 모델 구축
- 입력 데이터는 벡터로 레이블은 0과 1의 스칼라로 준비 됨. 
- 이런 유형의 모델은 층을 이루는 완전연결층(dense) 과 relu 활성화함수(activation function)을 통해 문제를 잘 해결할 수 있음
- 16개의 unit 을 갖춘 2개의 중간층과 3번째 층은 영화평의 긍정 부정을 나타낼 scalar 예측의 출력 층

### 모델 정의

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

### 모델 컴파일
- 궁극적으로 긍정과 부정을 판별하는 binary classification 문제로 출력의 확률분포에 대해 binary 분류를 하는데 binary_crossentropy loss 가 가장 좋은 선택
- crossentropy 는 ground-truth 분포와 모델 예측 값사이의 거리를 측정
- rmsprop 옵티마이저는 거의 모든 문제에 있어 좋은 default 선택
- accuracy 를 선언하여 훈련 동안 정확도를 모니터링

In [ ]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

## 모델 성능 검증
- train 데이터 중에 10000 개를 validate 용으로 별도 분할

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

## 모델 Training
- x_train 과 y_train tensor 를 20회 반복하면서 실행 (epoch)
- mini-batch 크기는 512 샘플
- 검증용으로 분할 했던 10000 개 샘플(x_val, y_val)에 대하여 정확도와 손실을 모니터

In [ ]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

- model.fit 은 History object 를 리턴. 
- 멤버로 가지고 있는 history 는 훈련 중에 일어나는 데이터 모든 것을 dictionary 형태로 보유
- dictionary 는 training 과 validation 의 accuracy 와 loss 를 각각 보유

In [ ]:
history_dict = history.history
history_dict.keys()

### Training 과 Validation loss 를 matplotlib 를 사용하여 plot

- 'bo' 는 blue dot 을 의미
- 'b'는 solid blue line 을 의미

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

### Training 과 Validation accuracy
- plt.clf 는 clear the picture

In [ ]:
plt.clf()
acc = history_dict["accuracy"]
val_acc = history_dict["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Training acc")
plt.plot(epochs, val_acc, "b", label="Validation acc")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

- training accuracy 는 지속적으로 증가하는 데 반해, validation accuracy 는 4 epoch 이후에 증가세가 멈춤
- 전형적인 overfitting 문제. 문제를 방지하기 위해 모델을 새롭게 4 epoch 까지만 정의하고 훈련 시행

In [ ]:
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

- 마지막 results 는 아래와 같음

In [ ]:
results

- 간단한 방법으로 88% 정확도를 달성
- state of art 방식의 접근법으로 95% 까지 달성 가능

### 훈련된 모델을 사용하여 새로운 데이터의 예측을 수행
- 결과에서 보듯이, 어떤 샘플에서는 0.99 이상 (긍정 표현) 혹은 0.01 이하 (부정 표현) 인 것도 있고, 자신하지 못하는  (0.6, 0.4) 샘플도 있다

In [ ]:
model.predict(x_test)

### Wrap up
- 신경망에 raw 데이터를 tensor 형태로 입력하기 전에 통상적으로 적지 않은 일련의 데이터 전처리 가 필요. 순차적인 단어들을 이진 벡터들로 인코딩 (one-hot-encoding). 다른 인코딩 방법들도 존재
- 몇개의 Dense 층과 함께 relu 활성화함수를 사용하여 넓은 범위의 문제들을 해결 (감성분석을 포함하여)할 수 있음. 
- 긍정, 부정과 같이 2개의 출력이 표시되는 binary 분류의 경우, 마지막 층이 Dense 층에 1개의 unit 을 갖고 sigmoid 활성화함수를 통해 scalar 의 0 과 1의 인코딩 확률 값을 출력
- 이러한 이진 분류의 scalar sigmoid 출력의 경우에 loss 함수는 binary_crossentropy 를 사용해야 함